# Computing the volume underneath a Pareto frontier
Given a Pareto-optimal frontier of solutions to a multi-criterion optimization model, this program computes the average distance from frontier points to the model's ideal solution. This provides a measure of the conflict among the objectives.

In [1]:
import pandas as pd
import numpy as np

### Preparation for volume computation algorithm

In [2]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("../solutionSets/3d/climateChange_EfficientSolutions_NoneOnly.csv")

In [3]:
# Hard-coded manipulations to tidy-up sols

sols = sols.drop(["SolutionIndex","Frontier"],axis=1) #remove solution index and frontier columns

In [4]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [5]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the sols dataframe

objSenses = {}

In [6]:
# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0 # all objectives are min for curr example
objSenses["Northern Spotted Owl Habitat (ha)"] = 1 #except owl

In [7]:
# Get the objectives' bounds

objBounds = {}
for obj,sense in objSenses.items():
    objBounds[obj] = [sols[obj].min(),sols[obj].max()]

In [8]:
# Get each objective's ideal value

idealObjVals = {}
for obj,bounds in objBounds.items():
    idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max

In [9]:
# Get each objective's worst value

worstObjVals = {}
for obj,bounds in objBounds.items():
    worstObjVals[obj] = bounds[not(objSenses[obj])]

In [10]:
# Compute the non-normalized volume of the objective space

objSpaceVolume_nonNorm = 1
for obj,bounds in objBounds.items():
    objSpaceVolume_nonNorm *= bounds[1] - bounds[0]

In [11]:
# Normalize the objective space by converting each value to the relative achievement along its axis:

#   distance from the worst case value
# ---------------------------------------
# total distance spanned by the objective

# Objective space is now in [0,1]^N where N is number of objectives


for obj,bounds in objBounds.items():
    sols[obj] = sols[obj].apply(
        lambda x: \
        abs(x - worstObjVals[obj]) / \
        (bounds[1]-bounds[0]))

### Computing the frontier's average distance to ideal solution

In [34]:
# Define the ideal solution

idealSolution = np.ones(len(objs))

In [35]:
# Initialize average distance to ideal

frontierAvgDist= 0

In [36]:
# Compute average distance

for solution in sols.itertuples(index=False,name=None):
    frontierAvgDist += np.linalg.norm(solution - idealSolution)
    
frontierAvgDist /= len(sols)